In [15]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import os

In [5]:
df=pd.read_csv('/opt/ml/input/data/all_feature_tabular_data.csv',index_col=0)
df.head(3)

,userID,modeTestId,accuracy,modeKT,modeKTCount,modeKTAcc,modeKTAccCate,modeKTCate,recAuccracyLast,modeBigClass,modeBigClassAccCate,modeBigClassAcc,userLVbyModeTest,modeTestLV,meanElapsedTime,meanElapsedTimeClass,dataset
0,0,818,0.630872,4784,14,0.266667,4,3,0.5,8,6,0.468320,37,5,36.057718,7.783893,1
1,1,341,0.853162,2010,35,0.972222,11,5,0.9,4,9,0.822719,58,4,80.610932,16.704180,1
2,2,209,0.612319,307,10,1.000000,11,6,0.3,3,10,0.876404,36,3,48.471014,10.311594,1


In [6]:
train=df[df['dataset']==1]
test=df[df['dataset']==2]
x_train=train.drop(columns=['accuracy'])
y_train=train['accuracy']
x_test=test.drop(columns=['accuracy'])
y_test=test['accuracy']

In [10]:
model = LGBMRegressor().fit(x_train, y_train)
predict = model.predict(x_test)
mean_squared_error(y_test, predict)

0.00427936651690123

In [14]:
submit=[[i,p] for i,p in enumerate(predict)]
submit_df=pd.DataFrame(data=submit, columns=['id','prediction'])
submit_df.head()

,id,prediction
0,0,0.644544
1,1,0.679175
2,2,0.725064
3,3,0.764228
4,4,0.733268


In [16]:
output_dir = "output/"
write_path = os.path.join(output_dir, "submission_tabular_LGBM.csv")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
submit_df.to_csv(write_path)

# 여러 epoch

In [23]:
import lightgbm as lgb
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action="ignore")

In [18]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_test = lgb.Dataset(x_test, y_test)

In [34]:
# hyper parameters
# TODO : tunning
params = {
    # "learning_rate": 0.01,
    # "max_depth": 8,
    # "boosting": "gbdt",  # rf, gbdt, dart, goss
    "objective": "regression",
    "metric": "rmse",
    # "num_leaves": 40,s
    # "feature_fraction": 0.8,
    # "bagging_fraction": 1,
    # "bagging_freq": 5,
    "seed": 42,
}

model = lgb.train(
    params,
    lgb_train,
    valid_sets=[lgb_train, lgb_test],
    verbose_eval=10,
    num_boost_round=10000,
    early_stopping_rounds=100,
)
preds = model.predict(x_test)
# acc = accuracy_score(y_test, np.where(preds >= 0.5, 1, 0))
# auc = roc_auc_score(y_test, preds)
# print(f"VALID AUC : {auc} ACC : {acc}\n")

print('test mean squared error : ',mean_squared_error(y_test, preds))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000884 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1873
[LightGBM] [Info] Number of data points in the train set: 6698, number of used features: 15
[LightGBM] [Info] Start training from score 0.628909
Training until validation scores don't improve for 100 rounds
[10]	training's rmse: 0.0878727	valid_1's rmse: 0.0921181
[20]	training's rmse: 0.0699795	valid_1's rmse: 0.0740336
[30]	training's rmse: 0.0644062	valid_1's rmse: 0.0689623
[40]	training's rmse: 0.061403	valid_1's rmse: 0.0670904
[50]	training's rmse: 0.0593167	valid_1's rmse: 0.0665333
[60]	training's rmse: 0.0576045	valid_1's rmse: 0.0661403
[70]	training's rmse: 0.0561783	valid_1's rmse: 0.0658675
[80]	training's rmse: 0.0547938	valid_1's rmse: 0.0657569
[90]	training's rmse: 0.053517	valid_1's rmse: 0.0656255
[100]	training's rmse: 0.0523822	valid_1's rmse: 0.0654169
[110]	training'

In [35]:
# MAKE PREDICTION
total_preds = model.predict(x_test)
total_preds

array([0.63967987, 0.69119907, 0.72623263, 0.76092784, 0.73450251,
       0.77647947, 0.66317049, 0.47248271, 0.36410657, 0.88615242,
       0.67750376, 0.76469643, 0.79643783, 0.72841853, 0.69867227,
       0.8117419 , 0.49456461, 0.70053865, 0.81118504, 0.51031994,
       0.83768997, 0.6886728 , 0.3122659 , 0.53083538, 0.74689856,
       0.6226188 , 0.73745844, 0.74896822, 0.65802789, 0.84190751,
       0.7849712 , 0.75437985, 0.78791088, 0.63644784, 0.77973457,
       0.69354614, 0.67859801, 0.7424788 , 0.70461356, 0.4202961 ,
       0.59774567, 0.63292791, 0.54818548, 0.35261307, 0.69128525,
       0.75882145, 0.65105145, 0.3427645 , 0.77944153, 0.64303882,
       0.71015963, 0.64075364, 0.63685221, 0.44515861, 0.79100283,
       0.68285686, 0.60050345, 0.78973795, 0.51930485, 0.61025487,
       0.76622735, 0.65746061, 0.72354287, 0.40345644, 0.59399322,
       0.58175616, 0.77208276, 0.60375394, 0.61613478, 0.79647776,
       0.74772305, 0.7830899 , 0.46298434, 0.61590256, 0.66645

In [38]:
len(total_preds[total_preds<0.5])

169

In [39]:
submit=[[i,p] for i,p in enumerate(total_preds)]
submit_df=pd.DataFrame(data=submit, columns=['id','prediction'])
submit_df.head()

,id,prediction
0,0,0.639680
1,1,0.691199
2,2,0.726233
3,3,0.760928
4,4,0.734503


In [40]:
output_dir = "output/"
write_path = os.path.join(output_dir, "submission_tabular_LGBM2.csv")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
submit_df.to_csv(write_path)